<a href="https://colab.research.google.com/github/ElenaMorozzova/NLP_Intro/blob/main/NLP_DZ6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -O imdb.zip -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1vrQ5czMHoO3pEnmofFskymXMkq_u1dPc"
!unzip imdb.zip
!pip -q install eli5
!pip -q install spacy
!python -m spacy download en

Archive:  imdb.zip
replace test.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: т
error:  invalid response [т]
replace test.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
     |████████████████████████████████| 12.0 MB 5.6 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [2]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id': '1d1-5FwxK53ePwygNWeG7jhsOWZbi5HOv'})
downloaded.GetContentFile('train_docs.pkl')

downloaded = drive.CreateFile({'id': '1MMOY477t965G0C5DtXeREVp0X85UaNq5'})
downloaded.GetContentFile('test_docs.pkl')

In [2]:
#Датасет взят с http://ai.stanford.edu/~amaas/data/sentiment/
!head train.tsv

is_positive	review
0	"Dreamgirls, despite its fistful of Tony wins in an incredibly weak year on Broadway, has never been what one would call a jewel in the crown of stage musicals. However, that is not to say that in the right cinematic hands it could not be fleshed out and polished into something worthwhile on-screen. Unfortunately, what transfers to the screen is basically a slavishly faithful version of the stage hit with all of its inherent weaknesses intact. First, the score has never been one of the strong points of this production and the film does not change that factor. There are lots of songs (perhaps too many?), but few of them are especially memorable. The closest any come to catchy tunes are the title song and One Night Only - the much acclaimed And I Am Telling You That I Am Not Going is less a great song than it is a dramatic set piece for the character of Effie (Jennifer Hudson). The film is slick and technically well-produced, but the story and characters are surprisi

**Анализ тональности текста**

Классификация отзывов с IMDB на положительные/отрицательные.

In [3]:
import pandas as pd

train_df = pd.read_csv("train.tsv", delimiter="\t")
test_df = pd.read_csv("test.tsv", delimiter="\t")

print('Train size = {}'.format(len(train_df)))
print('Test size = {}'.format(len(test_df)))

Train size = 25000
Test size = 25000


In [4]:
import re
#Избавимся от переносов <br />
pattern = re.compile('<br />')

print(train_df['review'].iloc[3])
print(pattern.subn(' ', train_df['review'].iloc[3])[0])

Spoilers ahead if you want to call them that...<br /><br />I would almost recommend this film just so people can truly see a 1/10. Where to begin, we'll start from the top...<br /><br />THE STORY: Don't believe the premise - the movie has nothing to do with abandoned cars, and people finially understanding what the mysterious happenings are. It's a draub, basic, go to cabin movie with no intensity or "effort".<br /><br />THE SCREENPLAY: I usually give credit to indie screenwriters, it's hard work when you are starting out...but this is crap. The story is flat - it leaves you emotionless the entire movie. The dialogue is extremely weak and predictable boasting lines of "Woah, you totally freaked me out" and "I was wondering if you'd uh...if you'd like to..uh, would you come to the cabin with me?". It makes me want to rip out all my hair, one strand at a time and feed it to myself.<br /><br />THE CHARACTERS: HOLY CRAP!!!! Some have described the characters as flat, I want to take it one 

In [5]:
train_df['review'] = train_df['review'].apply(lambda text: pattern.subn(' ', text)[0])
test_df['review'] = test_df['review'].apply(lambda text: pattern.subn(' ', text)[0])

In [19]:
#Используем countvectorizer для составления мешка слов и логистичекую регрессию для классификации
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

vectorizer = CountVectorizer()
classifier = LogisticRegression()

model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

In [7]:
model.fit(train_df['review'], train_df['is_positive'])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
             

In [8]:
from sklearn.metrics import accuracy_score
#Считаем accuracy модели
def eval_model(model, test_df):
    preds = model.predict(test_df['review'])
    print('Test accuracy = {:.2%}'.format(accuracy_score(test_df['is_positive'], preds)))
    
eval_model(model, test_df)

Test accuracy = 86.50%


In [9]:
#Посмотрим топ слов, которые влияют на классификатор (веса)
import eli5
eli5.show_weights(classifier, vec=vectorizer, top=40)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


Weight?,Feature
+1.855,refreshing
+1.760,wonderfully
+1.689,funniest
+1.647,surprisingly
+1.626,rare
+1.432,superb
+1.401,excellent
+1.365,incredible
+1.351,perfect
+1.302,delightful


In [10]:
#Посмотрим на примеры неправильной классификации
import numpy as np

preds = model.predict(test_df['review'])
incorrect_pred_index = np.random.choice(np.where(preds != test_df['is_positive'])[0])

eli5.show_prediction(classifier, test_df['review'].iloc[incorrect_pred_index],
                     vec=vectorizer, targets=['positive'], target_names=['negative', 'positive'])

Задание 1

Проверьте повысилось ли качество на стандартных подходах при лемматизации/и без неё

In [11]:
# import necessary libraries
import nltk

# word tokenization 
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Stemming
from nltk.stem import PorterStemmer
ps = PorterStemmer()

# Lemmatization
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

# stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
# Defining a function to perform tokenization, Stemming/Lemmatization and to remove stop words

def token_stem_stop(string):
    stem_rew = " "
    tokens = word_tokenize(string) # word tokenization
#     print(tokens)
    for word in tokens:
        if word.lower() not in stop_words: # removing stop words
            stem_word = ps.stem(word) # stemming
            stem_rew = stem_rew + " " + stem_word
    return str(stem_rew)

# apply the function on the dataframe and create a new column to store
train_df['review_processed'] = train_df['review'].apply(token_stem_stop) 
train_df.head() # showing the first five rows

,is_positive,review,review_processed
0,0,"Dreamgirls, despite its fistful of Tony wins i...","dreamgirl , despit fist toni win incred weak..."
1,0,This show comes up with interesting locations ...,show come interest locat fast travel channel...
2,1,I simply love this movie. I also love the Ramo...,"simpli love movi . also love ramon , sorta b..."
3,0,Spoilers ahead if you want to call them that.....,spoiler ahead want call ... would almost rec...
4,1,My all-time favorite movie! I have seen many m...,"all-tim favorit movi ! seen mani movi , one ..."


In [13]:
test_df['review_processed'] = test_df['review'].apply(token_stem_stop)

In [20]:
# Обучим линейную регрессию на обработанных отзывах
model.fit(train_df['review_processed'], train_df['is_positive'])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
             

In [21]:
#На обработанном лемматизированном столбце accuracy модели снизилась
#Считаем accuracy модели
def eval_model_pr(model, test_df):
    preds = model.predict(test_df['review_processed'])
    print('Test accuracy = {:.2%}'.format(accuracy_score(test_df['is_positive'], preds)))

eval_model_pr(model, test_df)

Test accuracy = 85.48%


Заменим CountVectorizer на TfidfVectorizer, чтобы учесть частотность слов и извлечем биграммы слов (параметр ngram_range)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1, 2))
classifier = LogisticRegression()

model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

# С TfidfVectorizer на необработанном столбце отзывов качество модели выше, чем с CountVectorizer 
model.fit(train_df['review'], train_df['is_positive'])
eval_model(model, test_df)

Test accuracy = 88.64%


In [23]:
# На обработанном столбце отзывов качество модели также чуть хуже
model.fit(train_df['review_processed'], train_df['is_positive'])
eval_model_pr(model, test_df)

Test accuracy = 87.84%


**Задание 2** 

В текстах рецензий очень много именованных сущностей. Удалите из текстов какие-то из сущностей, чтобы не придавать им окраску. Запустите классификатор.

In [24]:
import spacy
from spacy import displacy
nlp = spacy.load('en', disable=['parser'])

train_docs = [doc for doc in nlp.pipe(train_df.review.values)]
test_docs = [doc for doc in nlp.pipe(test_df.review.values)]

In [25]:
displacy.render(train_docs[0], style='ent', jupyter=True)

In [26]:
from tqdm.notebook import tqdm

no_ent_train = []
for i, doc in tqdm(enumerate((train_docs))):
    plain_text=''
    start_point=0
    for ent in doc.ents:
        plain_text = plain_text+train_df.iloc[i]['review'][start_point : ent.start_char]+' '+ent.label_+' '
        start_point = ent.end_char   
    no_ent_train.append(plain_text)

0it [00:00, ?it/s]

In [27]:
no_ent_test = []
for i, doc in tqdm(enumerate((test_docs))):
    plain_text=''
    start_point=0
    for ent in doc.ents:
        plain_text = plain_text+test_df.iloc[i]['review'][start_point : ent.start_char]+' '+ent.label_+' '
        start_point = ent.end_char   
    no_ent_test.append(plain_text)

0it [00:00, ?it/s]

In [28]:
# adding column
train_df['no_ent'] = no_ent_train
test_df['no_ent']  = no_ent_test

In [29]:
test_df.head()

,is_positive,review,review_processed,no_ent
0,0,A formulaic story with all the tired cliches. ...,formula stori tire clich . shock horribl scr...,A formulaic story with all the tired cliches. ...
1,1,This is both an entertaining and a touching ve...,"entertain touch version classic tale , also ...",This is both an entertaining and a touching ve...
2,0,Corey Haim is never going to be known as one o...,corey haim never go known one great actor ti...,PERSON is never going to be known as CARDIN...
3,1,If you believe that any given war movie can ma...,"believ given war movi make realli feel war ,...",If you believe that any given war movie can ma...
4,1,Carla Gugino literally melts the screen in thi...,carla gugino liter melt screen crime caper ....,PERSON literally melts the screen in this cr...


In [30]:
# Logistic regression classificator train
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
classifier = LogisticRegression()

model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

model.fit(train_df['no_ent'], train_df['is_positive'])

Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, inter

In [31]:
# Evaluating
eval_model(model, test_df)

Test accuracy = 88.28%


Обучим нейронную сеть в качестве модели классификации

In [32]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Dropout
from collections import Counter

tf.__version__

'2.6.0'

In [33]:

# Пронумеруем слова и конвертируем данные в array
words_counter = Counter((word for text in train_df.review for word in text.lower().split()))

word2idx = {
    '': 0,
    '<unk>': 1
}
for word, count in words_counter.most_common():
    if count < 10:
        break
        
    word2idx[word] = len(word2idx)
    
print('Words count', len(word2idx))

Words count 26783


In [34]:
def convert(texts, word2idx, max_text_len):
    data = np.zeros((len(texts), max_text_len), dtype=np.int)
    
    for inx, text in enumerate(texts):
        result = []
        for word in text.split():
            if word in word2idx:
                result.append(word2idx[word])
        padding = [0]*(max_text_len - len(result))
        data[inx] = np.array(padding + result[-max_text_len:], dtype=np.int)
    return data

X_train = convert(train_df.review, word2idx, 1000)
X_test = convert(test_df.review, word2idx, 1000)

In [35]:
X_train

array([[    0,     0,     0, ...,    27,  2173,    14],
       [    0,     0,     0, ...,    43,     2,  3361],
       [    0,     0,     0, ...,    86,  1326,   106],
       ...,
       [    0,     0,     0, ...,    12,     7, 17818],
       [    0,     0,     0, ...,    73,    27,  3112],
       [    0,     0,     0, ...,    28,    10, 23594]])

In [36]:
dim = int(len(word2idx)**(1/2))

# Используем сверточную нейронную сеть с binary_crossentropy, т.к. это бинарная классификация
model = Sequential([
    Embedding(input_dim=len(word2idx), output_dim=dim, input_shape=(X_train.shape[1],)),
    GlobalMaxPooling1D(),
    Dense(units=10, activation='relu'),
    Dense(units=10, activation='relu'),
    Dense(units=1, activation='sigmoid')
])

model.summary()
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 163)         4365629   
_________________________________________________________________
global_max_pooling1d (Global (None, 163)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1640      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 4,367,390
Trainable params: 4,367,390
Non-trainable params: 0
_________________________________________________________________


In [37]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train, train_df.is_positive, batch_size=128, epochs=5, 
          validation_data=(X_test, test_df.is_positive), callbacks=[callback])

Epoch 1/5
196/196 [==============================] - 40s 199ms/step - loss: 0.6103 - accuracy: 0.6858 - val_loss: 0.4349 - val_accuracy: 0.8506
Epoch 2/5
196/196 [==============================] - 39s 201ms/step - loss: 0.2911 - accuracy: 0.8928 - val_loss: 0.2827 - val_accuracy: 0.8826
Epoch 3/5
196/196 [==============================] - 39s 199ms/step - loss: 0.1558 - accuracy: 0.9464 - val_loss: 0.2786 - val_accuracy: 0.8843
Epoch 4/5
196/196 [==============================] - 39s 199ms/step - loss: 0.0760 - accuracy: 0.9810 - val_loss: 0.3036 - val_accuracy: 0.8808
Epoch 5/5
196/196 [==============================] - 39s 199ms/step - loss: 0.0319 - accuracy: 0.9945 - val_loss: 0.3392 - val_accuracy: 0.8802


In [38]:
# accuracy логистической регресси = 88.3% несколько выше, чем в модели простой нейронной сети
model.evaluate(X_test, test_df.is_positive)

782/782 [==============================] - 5s 6ms/step - loss: 0.3392 - accuracy: 0.8802


[0.3392295241355896, 0.8801599740982056]

In [39]:
# Обучим модель на обработанном лемматизированном столбце
X_train = convert(train_df.review_processed, word2idx, 1000)
X_test = convert(test_df.review_processed, word2idx, 1000)

In [40]:
model.fit(X_train, train_df.is_positive, batch_size=128, epochs=5, 
          validation_data=(X_test, test_df.is_positive), callbacks=[callback])

Epoch 1/5
196/196 [==============================] - 38s 196ms/step - loss: 0.3114 - accuracy: 0.8686 - val_loss: 0.3601 - val_accuracy: 0.8437
Epoch 2/5
196/196 [==============================] - 38s 196ms/step - loss: 0.1770 - accuracy: 0.9363 - val_loss: 0.3942 - val_accuracy: 0.8389
Epoch 3/5
196/196 [==============================] - 39s 197ms/step - loss: 0.1150 - accuracy: 0.9648 - val_loss: 0.4249 - val_accuracy: 0.8390
Epoch 4/5
196/196 [==============================] - 39s 197ms/step - loss: 0.0694 - accuracy: 0.9832 - val_loss: 0.4763 - val_accuracy: 0.8326


In [41]:
# accuracy получилось даже ниже, чем при обучении модели на исходных отзывах
model.evaluate(X_test, test_df.is_positive)

782/782 [==============================] - 5s 7ms/step - loss: 0.3601 - accuracy: 0.8437


[0.36012032628059387, 0.843720018863678]